In [1]:
import os 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)
import sys
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("load_explore") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 09:05:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# loads relevant datas in DataFrames
train_sessions_engineered = spark.read.csv('../Data/session_engineered_features.txt',header=False,
                                          inferSchema=True)


train_purchases = spark.read.load('../Data/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [7]:
train_purchases.printSchema()
#train_sessions_engineered.printSchema()

root
 |-- session_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- date: string (nullable = true)



In [ ]:
# INPUT = RDD with already engineered features
# two columns of interest MOST VISITED ITEM (clustered) and ITEM MOST TIME SPENT ON (clustered)
# the idea is to find a the most bought item (clustered) for each of these value
# we'll use the mapping item-id -- cluster (n=50) previously made on the categories features

bought_cluster = train_purchases.rdd.join(clustered_items)

# join the features df and 
temp_features = train_sessions_engineered.rdd.join(bought_cluster)



In [ ]:
def mode_and_least(row):
    vals, counts = np.unique(np.array(row), return_counts=True)
    mode_value = np.argwhere(counts == np.max(counts))[0][0]
    least_value = np.argwhere(counts == np.min(counts))[0][0]
    return (row[mode_value],row[least_value])
    
temp_most_visited = temp_features.rdd.map(lambda x : (x[4],(x[41],))) # keep only most_visited cluster and bought cluster
temp_most_visited = temp_most_visited.reduceByKey(lambda x,y : (x+y))
temp_most_visited = temp_most_visited.map(lambda x : (x[0], mode_and_least(x[1]))) # mapping : most visited clusted id -->TO--> most and least bought (cluster)

In [25]:
import numpy as np
def mode_and_least(row):
    vals, counts = np.unique(np.array(row), return_counts=True)
    mode_value = np.argwhere(counts == np.max(counts))[0][0]
    least_value = np.argwhere(counts == np.min(counts))[0][0]
    return (row[mode_value],row[least_value])
row = (1,4,5,7,87,9,5,1,4,5,6,7,8,2,4,5,6,6,6,8,7,1,1,2,3,4,8,1,1,1,1)
a = mode_and_least(row)

In [26]:
a

(1, 5)

In [27]:
row+(1,2)

(1,
 4,
 5,
 7,
 87,
 9,
 5,
 1,
 4,
 5,
 6,
 7,
 8,
 2,
 4,
 5,
 6,
 6,
 6,
 8,
 7,
 1,
 1,
 2,
 3,
 4,
 8,
 1,
 1,
 1,
 1,
 1,
 2)